<a href="https://colab.research.google.com/github/ayushi15092002/mental-health-workload/blob/main/Feature_Selection_Hybrid_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
from sklearn.feature_selection import RFE
# from feature_selection import hybrid


## Load Dataset

In [ ]:
data = pd.read_excel(r'/content/drive/My Drive/drdo/features/min_max_norm.xlsx')

In [ ]:
data.head(5)

,Coeffiecient_of_Variation,Mean_of_Vertex_to_Vertex_Slope,Variance_of_Vertex_to_Vertex_Slope,Hjorth_Activity,Hjorth_Mobility,Hjorth_Complexity,Kurtosis,2nd_Difference_Mean,2nd_Difference_Max,Skewness,...,28_Wavelet_Approximate_Std_Deviation,28_Wavelet_Approximate_Energy,28_Wavelet_Detailed_Mean,28_Wavelet_Detailed_Std _Deviation,28_Wavelet_Detailed_Energy,28_Wavelet_Approximate_Entropy,28_Wavelet_Detailed_Entropy,Hurst_Exponent,Permutation_Entropy,y
0,0.534439,0.614380,0.096161,0.020099,0.555111,0.052773,0.159801,0.248315,0.320428,0.528456,...,0.103340,0.009270,0.384685,0.344930,0.171401,0.184330,0.011559,0.723232,0.818775,1.0
1,0.534315,0.649929,0.068544,0.025495,0.455425,0.084819,0.133192,0.202098,0.254608,0.543443,...,0.146594,0.008348,0.571264,0.323252,0.155246,0.166751,0.010475,0.728989,0.812935,1.0
2,0.534344,0.583582,0.112827,0.027777,0.494449,0.070979,0.099703,0.281429,0.347117,0.595850,...,0.081346,0.004189,0.484646,0.452760,0.262473,0.278811,0.005396,0.751985,0.878779,1.0
3,0.542825,0.572106,0.097661,0.033724,0.452482,0.085299,0.141227,0.260114,0.302845,0.450263,...,0.141749,0.008017,0.776456,0.437634,0.248645,0.263571,0.010023,0.590368,0.849525,1.0
4,0.534477,0.629300,0.097666,0.029060,0.475090,0.078828,0.121195,0.253848,0.293299,0.568707,...,0.119258,0.008177,0.550035,0.382984,0.201500,0.215768,0.010123,0.612329,0.855054,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['y'], axis=1), 
                                                    data.y, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((9256, 246), (2314, 246))

##  Recursive Feature Elimination 
### with Random Forests Importance


### Example 1
This method is slightly **different from the guide**, as it use a different stopping criterion: the desired number of features to select is eventually reached.

In [ ]:
#  n_features_to_select decide the stopping criterion
# we stop till 10 features remaining

sel_ = RFE(RandomForestClassifier(n_estimators=20), n_features_to_select=10)
sel_.fit(X_train.fillna(0), y_train)

RFE(estimator=RandomForestClassifier(n_estimators=20), n_features_to_select=10)

In [ ]:
selected_feat = X_train.columns[(sel_.get_support())]
print(selected_feat)

Index(['Delta/Theta', '7_Wavelet_Detailed_Energy', '8_Wavelet_Detailed_Energy',
       '10_Wavelet_Approximate_Entropy', '12_Wavelet_Detailed_Std _Deviation',
       '20_Wavelet_Approximate_Entropy', '21_Wavelet_Approximate_Entropy',
       '23_Wavelet_Detailed_Energy', '28_Wavelet_Approximate_Entropy',
       'Permutation_Entropy'],
      dtype='object')


### Example 2
recursive feature elimination with RandomForest
with the method same as the guide
1. Rank the features according to their importance derived from a machine learning algorithm: it can be tree importance, or LASSO / Ridge,  or the linear / logistic regression coefficients.
2. Remove one feature -the least important- and build a machine learning algorithm utilizing the remaining features.
3. Calculate a performance metric of your choice: roc-auc, mse, rmse, accuracy.
4. If the metric decreases by more of an arbitrarily set threshold, then that feature is important and should be kept. Otherwise, we can remove that feature.
5. Repeat steps 2-4 until all features have been removed (and therefore evaluated) and the drop in performance assessed.


In [ ]:
# tol decide whether we should drop or keep the feature in current round
features_to_keep = recursive_feature_elimination_rf(X_train=X_train,
                                                           y_train=y_train,
                                                           X_test=X_test,
                                                           y_test=y_test,
                                                           tol=0.001)


testing feature:  Coeffiecient_of_Variation  which is feature  1  out of  246
New Test ROC AUC=0.7368984378640789
All features Test ROC AUC=0.7503746387547208
Drop in ROC AUC=0.013476200890641832
keep:  Coeffiecient_of_Variation

testing feature:  Mean_of_Vertex_to_Vertex_Slope  which is feature  2  out of  246
New Test ROC AUC=0.7498865052479988
All features Test ROC AUC=0.7503746387547208
Drop in ROC AUC=0.0004881335067219439
remove:  Mean_of_Vertex_to_Vertex_Slope

testing feature:  Variance_of_Vertex_to_Vertex_Slope  which is feature  3  out of  246
New Test ROC AUC=0.7602952929735481
All features Test ROC AUC=0.7498865052479988
Drop in ROC AUC=-0.010408787725549318
remove:  Variance_of_Vertex_to_Vertex_Slope

testing feature:  Hjorth_Activity  which is feature  4  out of  246
New Test ROC AUC=0.7435921568394672
All features Test ROC AUC=0.7602952929735481
Drop in ROC AUC=0.01670313613408092
keep:  Hjorth_Activity

testing feature:  Hjorth_Mobility  which is feature  5  out of  24

In [ ]:
features_to_keep

['Coeffiecient_of_Variation',
 'Hjorth_Activity',
 'Hjorth_Mobility',
 'Hjorth_Complexity',
 'Kurtosis',
 '2nd_Difference_Mean',
 '2nd_Difference_Max',
 'Skewness',
 '1st_Difference_Mean',
 '1st_Difference_Max',
 'FFT_Delta_MaxPower',
 'FFT_Theta_MaxPower',
 'FFT_Alpha_MaxPower',
 'FFT_Beta_MaxPower',
 'Delta/Theta',
 'Delta/Alpha',
 'Theta/Alpha',
 '(Delta+Theta)/Alpha',
 '1_Wavelet_Approximate_Mean',
 '1_Wavelet_Approximate_Std_Deviation',
 '1_Wavelet_Approximate_Energy',
 '1_Wavelet_Detailed_Mean',
 '1_Wavelet_Detailed_Std_Deviation',
 '1_Wavelet_Detailed_Energy',
 '1_Wavelet_Approximate_Entropy',
 '1_Wavelet_Detailed_Entropy',
 '2_Wavelet_Approximate_Mean',
 '2_Wavelet_Approximate_Std_Deviation',
 '2_Wavelet_Approximate_Energy',
 '2_Wavelet_Detailed_Mean',
 '2_Wavelet_Detailed_Std _Deviation',
 '2_Wavelet_Detailed_Energy',
 '2_Wavelet_Approximate_Entropy',
 '2_Wavelet_Detailed_Entropy',
 '3_Wavelet_Approximate_Mean',
 '3_Wavelet_Approximate_Std_Deviation',
 '3_Wavelet_Approximate_E

## Recursive Feature Addition
### with Random Forests Importance

### Example 1
recursive feature addition with RandomForest
with the method same as the guide
1. Rank the features according to their importance derived from a  machine learning algorithm: it can be tree importance, or LASSO / Ridge,  or the linear / logistic regression coefficients.
2. Build a machine learning model with only 1 feature, the most important one, and calculate the model metric for performance.
3. Add one feature -the most important- and build a machine learning  algorithm utilizing the added and any feature from previous rounds.
4. Calculate a performance metric of your choice: roc-auc, mse, rmse, accuracy.
5. If the metric increases by more than an arbitrarily set threshold,  then that feature is important and should be kept. Otherwise, we can  remove that feature.
6. Repeat steps 2-5 until all features have been removed (and therefore evaluated) and the drop in performance assessed.


In [ ]:
features_to_keep = recursive_feature_addition_rf(X_train=X_train,
                                                        y_train=y_train,
                                                        X_test=X_test,
                                                        y_test=y_test,
                                                        tol=0.001)


testing feature:  Mean_of_Vertex_to_Vertex_Slope  which is feature  1  out of  246
New Test ROC AUC=0.5035143245808468
All features Test ROC AUC=0.49189948724650573
Increase in ROC AUC=0.011614837334341044
keep:  Mean_of_Vertex_to_Vertex_Slope

testing feature:  Variance_of_Vertex_to_Vertex_Slope  which is feature  2  out of  246
New Test ROC AUC=0.5111411310726159
All features Test ROC AUC=0.5035143245808468
Increase in ROC AUC=0.007626806491769167
keep:  Variance_of_Vertex_to_Vertex_Slope

testing feature:  Hjorth_Activity  which is feature  3  out of  246
New Test ROC AUC=0.5334201088688635
All features Test ROC AUC=0.5111411310726159
Increase in ROC AUC=0.022278977796247545
keep:  Hjorth_Activity

testing feature:  Hjorth_Mobility  which is feature  4  out of  246
New Test ROC AUC=0.5549981535512211
All features Test ROC AUC=0.5334201088688635
Increase in ROC AUC=0.021578044682357578
keep:  Hjorth_Mobility

testing feature:  Hjorth_Complexity  which is feature  5  out of  246
New 

In [ ]:
features_to_keep

['Coeffiecient_of_Variation',
 'Mean_of_Vertex_to_Vertex_Slope',
 'Variance_of_Vertex_to_Vertex_Slope',
 'Hjorth_Activity',
 'Hjorth_Mobility',
 'Hjorth_Complexity',
 'Kurtosis',
 '2nd_Difference_Mean',
 '2nd_Difference_Max',
 '1st_Difference_Mean',
 'FFT_Delta_MaxPower',
 'FFT_Theta_MaxPower',
 'FFT_Alpha_MaxPower',
 'FFT_Beta_MaxPower',
 '1_Wavelet_Detailed_Std_Deviation',
 '1_Wavelet_Detailed_Energy',
 '2_Wavelet_Detailed_Std _Deviation',
 '2_Wavelet_Detailed_Energy',
 '3_Wavelet_Approximate_Mean',
 '3_Wavelet_Detailed_Std _Deviation',
 '3_Wavelet_Detailed_Energy',
 '4_Wavelet_Detailed_Std _Deviation',
 '5_Wavelet_Detailed_Std _Deviation',
 '5_Wavelet_Detailed_Energy',
 '5_Wavelet_Approximate_Entropy',
 '7_Wavelet_Detailed_Std _Deviation',
 '7_Wavelet_Detailed_Energy',
 '8_Wavelet_Detailed_Std _Deviation',
 '8_Wavelet_Detailed_Energy',
 '10_Wavelet_Detailed_Mean',
 '12_Wavelet_Detailed_Std _Deviation',
 '12_Wavelet_Detailed_Energy',
 '13_Wavelet_Detailed_Std _Deviation',
 '14_Wavele

In [ ]:
from sklearn.ensemble import RandomForestClassifier #, RandomForestRegressor
from sklearn.metrics import roc_auc_score #, mean_squared_error

# 2018.12.02 Created by Eamon.Zhang


def recursive_feature_elimination_rf(X_train,y_train,X_test,y_test,
                                     tol=0.001,max_depth=None,
                                     class_weight=None,
                                     top_n=15,n_estimators=50,random_state=0):
    
   
    features_to_remove = []
    count = 1
    # initial model using all the features
    model_all_features = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                    random_state=random_state,class_weight=class_weight,
                                    n_jobs=-1)
    model_all_features.fit(X_train, y_train)
    y_pred_test = model_all_features.predict_proba(X_test)
    auc_score_all = roc_auc_score(y_test, y_pred_test,multi_class = 'ovr', average = 'weighted')
    
    for feature in X_train.columns:
        print()
        print('testing feature: ', feature, ' which is feature ', count,
          ' out of ', len(X_train.columns))
        count += 1
        model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                    random_state=random_state,class_weight=class_weight,
                                    n_jobs=-1)
        
        # fit model with all variables minus the removed features
        # and the feature to be evaluated
        model.fit(X_train.drop(features_to_remove + [feature], axis=1), y_train)
        y_pred_test = model.predict_proba(
                    X_test.drop(features_to_remove + [feature], axis=1))   
        auc_score_int = roc_auc_score(y_test, y_pred_test,multi_class = 'ovr', average = 'weighted')
        print('New Test ROC AUC={}'.format((auc_score_int)))
    
        # print the original roc-auc with all the features
        print('All features Test ROC AUC={}'.format((auc_score_all)))
    
        # determine the drop in the roc-auc
        diff_auc = auc_score_all - auc_score_int
    
        # compare the drop in roc-auc with the tolerance
        if diff_auc >= tol:
            print('Drop in ROC AUC={}'.format(diff_auc))
            print('keep: ', feature)
            
        else:
            print('Drop in ROC AUC={}'.format(diff_auc))
            print('remove: ', feature)
            
            # if the drop in the roc is small and we remove the
            # feature, we need to set the new roc to the one based on
            # the remaining features
            auc_score_all = auc_score_int
            
            # and append the feature to remove to the list
            features_to_remove.append(feature)
    print('DONE!!')
    print('total features to remove: ', len(features_to_remove))  
    features_to_keep = [x for x in X_train.columns if x not in features_to_remove]
    print('total features to keep: ', len(features_to_keep))
    
    return features_to_keep


def recursive_feature_addition_rf(X_train,y_train,X_test,y_test,
                                     tol=0.001,max_depth=None,
                                     class_weight=None,
                                     top_n=15,n_estimators=50,random_state=0):
    
   
    features_to_keep = [X_train.columns[0]]
    count = 1
    # initial model using only one feature
    model_one_feature = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                    random_state=random_state,class_weight=class_weight,
                                    n_jobs=-1)
    model_one_feature.fit(X_train[[X_train.columns[0]]], y_train)
    y_pred_test = model_one_feature.predict_proba(X_test[[X_train.columns[0]]]) 
    auc_score_all = roc_auc_score(y_test, y_pred_test,multi_class = 'ovr', average = 'weighted')
    
    for feature in X_train.columns[1:]:
        print()
        print('testing feature: ', feature, ' which is feature ', count,
          ' out of ', len(X_train.columns))
        count += 1
        model = RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,
                                    random_state=random_state,class_weight=class_weight,
                                    n_jobs=-1)
        
        # fit model with  the selected features
        # and the feature to be evaluated
        model.fit(X_train[features_to_keep + [feature]], y_train)
        y_pred_test = model.predict_proba(
                    X_test[features_to_keep + [feature]])  
        auc_score_int = roc_auc_score(y_test, y_pred_test )
        print('New Test ROC AUC={}'.format((auc_score_int)))
    
        # print the original roc-auc with all the features
        print('All features Test ROC AUC={}'.format((auc_score_all)))
    
        # determine the drop in the roc-auc
        diff_auc = auc_score_int - auc_score_all
    
        # compare the drop in roc-auc with the tolerance
        if diff_auc >= tol:
            # if the increase in the roc is bigger than the threshold
            # we keep the feature and re-adjust the roc-auc to the new value
            # considering the added feature
            print('Increase in ROC AUC={}'.format(diff_auc))
            print('keep: ', feature)
            auc_score_all = auc_score_int
            features_to_keep.append(feature)
        else:
            print('Increase in ROC AUC={}'.format(diff_auc))
            print('remove: ', feature)          

    print('DONE!!')
    print('total features to keep: ', len(features_to_keep))  
   
    return features_to_keep